# Ben E Keith COVID Analysis

Goal: 
1. Process BEK data (source = data.attribytes.com)
2. Analyze data using COVID segmentation
3. Compare sell-out (Ben E Keith) to sell-in (McCain) data

### 1. Load libraries, initiate folder/file paths
Run cell below

In [1]:
import pandas as pd
import datetime
from datetime import datetime as dt
import numpy as np
import teradatasql

#path where dictionary file can be found
#Neil
DICTIONARY = r'C:\Users\NEWATTER\OneDrive - McCain Foods Limited\Distributor Sell-Out Dictionaries\\'
#Joe
#DICTIONARY = r'C:\Users\jcronk\McCain Foods Limited\GNA Data Strategy & Analytics - COVID Recovery\Distributor Sell-Out Dictionaries\\'

#main path
#Neil
PATH = r'C:\Users\NEWATTER\OneDrive - McCain Foods Limited\Historical Sell-Out Sales\\'
#Joe
#PATH = r'C:\Users\jcronk\McCain Foods Limited\GNA Data Strategy & Analytics - COVID Recovery\Historical Sell-Out Sales\\'

#backup path
#Neil
BACKUP = r'C:\Users\NEWATTER\OneDrive - McCain Foods Limited\Historical Sell-Out Sales\Backups\\'
#Joe
#BACKUP = r'C:\Users\jcronk\McCain Foods Limited\GNA Data Strategy & Analytics - COVID Recovery\Historical Sell-Out Sales\Backups\\'

#time dataframe
TIME = pd.read_excel(DICTIONARY + 'Time Definitions.xlsx')

### 2. Data Dictionary
Run cell below

In [2]:
def us_states():
    us_state_abbrev = {
        'Alabama': 'AL',
        'Alaska': 'AK',
        'American Samoa': 'AS',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Guam': 'GU',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Northern Mariana Islands':'MP',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Puerto Rico': 'PR',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virgin Islands': 'VI',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'
    }

    # thank you to @kinghelix and @trevormarburger for this idea
    abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))
    
    return pd.DataFrame.from_dict(abbrev_us_state, orient = 'index', columns = ['State Name']).rename_axis('State').reset_index()


def apply_dictionary(df, file_name):
    
    #create dictionary object from Excel file
    #adding sheet_name = None makes it a dictionary type
    _dict = pd.read_excel(DICTIONARY + file_name, sheet_name = None, engine='openpyxl')
    
    #create DataFrame from dictionary object called dict (short for dictionary)
    dict_df = pd.DataFrame.from_dict(_dict['Segment Mapping'])
    
    #create DataFrame from dictionary object called cat (short for category)
    sku_df = pd.DataFrame.from_dict(_dict['SKU Mapping'])
    
    #print shape of df (dimensions)
    print(f'Shape before adding dictionary: {df.shape}', flush = True)

    #Business Unit	SIC Code	SIC Sub
    #Group
    
    #add lower case for merging
    dict_df['Business Unit-lower'] = dict_df['Business Unit'].str.lower()
    dict_df['SIC Code-lower'] = dict_df['SIC Code'].str.lower()
    dict_df['SIC Sub-lower'] = dict_df['SIC Sub'].str.lower()
    
    #COVID Segmentation - L1	COVID Segmentation - L2	COVID Segmentation - (Restaurants)	COVID Segmentation - (Restaurants: Sub-Segment)	Restaurant Service Type	Cuisine Type
    dict_df = dict_df.groupby(['COVID Segmentation - L1','Business Unit-lower','SIC Code-lower','SIC Sub-lower',
                               'COVID Segmentation - L2','COVID Segmentation - (Restaurants)','COVID Segmentation - (Restaurants: Sub-Segment)',
                               'Restaurant Service Type','Cuisine Type'], dropna = False).size().reset_index().drop(columns={0})
    
    #add lower case key columns for merging (removes case mismatch)
    df['Business Unit-lower'] = df['Business Unit'].str.lower()
    df['SIC Code-lower'] = df['SIC Code'].str.lower()
    df['SIC Sub-lower'] = df['SIC Sub'].str.lower()
    
    df = df.rename(columns = {
        'Customer City':'City', 
        'Customer State':'State',
        'Manufacture Prod.Nbr.':'SKU ID'
    })
    
    df = df.merge(dict_df, how = 'left', left_on = ['Business Unit-lower','SIC Code-lower','SIC Sub-lower'],
                  right_on = ['Business Unit-lower','SIC Code-lower','SIC Sub-lower']).drop(columns = {'Business Unit-lower','SIC Code-lower','SIC Sub-lower'})
    
    #print shape of df (dimensions)
    print(f'Shape after adding segmentation: {df.shape}', flush = True)
    
    #SKU Mapping
    
    df['SKU ID'] = df['SKU ID'].astype(str)
    sku_df['Mfg ID'] = sku_df['Mfg ID'].astype(str)
    
    df['Mfg ID-lower'] = df['SKU ID'].str.lower()
    sku_df['Mfg ID-lower'] = sku_df['Mfg ID'].str.lower()
    
    sku_df = sku_df.groupby(['Mfg ID-lower','Consolidated Category','L1 Product Hierarchy','L2 Product Hierarchy'], dropna = False).size().reset_index().drop(columns={0})
    
    df = df.merge(sku_df, how = 'left', left_on = ['Mfg ID-lower'],right_on = ['Mfg ID-lower']).drop(columns = {'Mfg ID-lower'})
    
    #print shape of df (dimensions)
    print(f'Shape after adding product segmentation: {df.shape}', flush = True)
    
    df['Week Starting'] = pd.to_datetime(df['Week of'])
    
    print(f'Shape before adding time: {df.shape}', flush = True)
    
    df = df.merge(TIME[['Week Starting (Sun)', 'Calendar Week Year']], how = 'left', 
                  left_on = ['Week Starting'], right_on = ['Week Starting (Sun)']).drop(columns = {'Week Starting (Sun)'})
    
    print(f'Shape after adding time: {df.shape}', flush = True)
    
    #exclude certain records
    df = df[~df['Calendar Week Year'].isna()]
    df = df[df['Branch'] != 'Total']
    df['LBS'] = pd.to_numeric(df['LBS'])
    df['Calendar Week Year'] = df['Calendar Week Year'].astype('int64')
    
    #Merge states
    df = df.merge(us_states(), how = 'left', on = 'State')
       
    df = clean_city(df)
    
    print(f'Shape after adding dictionary: {df.shape}', flush = True)
    
    return df

### 3. Import File
Run cell below

In [3]:
def import_file(file_name):
    
#import file
    if '.csv' in file_name:
        df = pd.read_csv(file_name, thousands = ',', encoding="utf-8", low_memory = False, header = 0,na_values = " ")
        df = df[df['Branch'] != 'Total']
        #df['Unnamed: 21'] = df['Unnamed: 21'].replace('[\$,)]','', regex=True).replace('[(]','-', regex=True).astype(float)
    else:
        df = pd.DataFrame()
        
        _import = pd.read_excel(file_name, sheet_name=None)
        
        for f in _import:
            print(f)
            if f == 'Sheet1':
                add = pd.DataFrame.from_dict(_import[f])
                
                col = add.columns.to_list()
            else:
                add = _import[f].T.reset_index().T
                add.columns = col
        
            df = df.append(add)
            
    return df.rename(columns={
        'Unnamed: 21':'LBS',
        'Unnamed: 22':'LBS'
    })

### 4. Calculation Functions
Run cell below

In [4]:
def add_rolling(df, _list):
    #groupby _list
    df = df.groupby(_list, dropna = False)[['LBS','LBS_LY','LBS_Baseline']].sum().reset_index()
    
    #set index to all but last column in list
    df = df.set_index(_list)
    
    #add new metric SMA_4 (simple moving average - 4 periods)
    #level = all but last 2 items in list
    df['LBS_Lag_1'] = df.groupby(level=_list[0:-1])['LBS'].shift(periods = 1)
    df['LBS_Lag_2'] = df.groupby(level=_list[0:-1])['LBS'].shift(periods = 2)
    df['LBS_Lag_3'] = df.groupby(level=_list[0:-1])['LBS'].shift(periods = 3)
    df['LBS_Lag_4'] = df.groupby(level=_list[0:-1])['LBS'].shift(periods = 4)
    
    df['SMA_4'] = df.groupby(level=_list[0:-1])['LBS'].apply(lambda x: x.rolling(4, min_periods=1).mean())
    df['SMA_8'] = df.groupby(level=_list[0:-1])['LBS'].apply(lambda x: x.rolling(8, min_periods=1).mean())
    df['SMA_12'] = df.groupby(level=_list[0:-1])['LBS'].apply(lambda x: x.rolling(12, min_periods=1).mean())
    
    df['SMA_4_LY'] = df.groupby(level=_list[0:-1])['LBS_LY'].apply(lambda x: x.rolling(4, min_periods=1).mean())
    df['SMA_8_LY'] = df.groupby(level=_list[0:-1])['LBS_LY'].apply(lambda x: x.rolling(8, min_periods=1).mean())
    df['SMA_12_LY'] = df.groupby(level=_list[0:-1])['LBS_LY'].apply(lambda x: x.rolling(12, min_periods=1).mean())
    
    df['SMA_4_Baseline'] = df.groupby(level=_list[0:-1])['LBS_Baseline'].apply(lambda x: x.rolling(4, min_periods=1).mean())
    df['SMA_8_Baseline'] = df.groupby(level=_list[0:-1])['LBS_Baseline'].apply(lambda x: x.rolling(8, min_periods=1).mean())
    df['SMA_12_Baseline'] = df.groupby(level=_list[0:-1])['LBS_Baseline'].apply(lambda x: x.rolling(12, min_periods=1).mean())
    
    df['LBS_Baseline_Lag_1'] = df.groupby(level=_list[0:-1])['LBS_Baseline'].shift(periods = 1)
    df['LBS_LY_Lag_1'] = df.groupby(level=_list[0:-1])['LBS'].shift(periods = 1)
    
    df['SMA_4_Lag_1'] = df.groupby(level=_list[0:-1])['SMA_4'].shift(periods = 1)
    df['SMA_4_LY_Lag_1'] = df.groupby(level=_list[0:-1])['SMA_4_LY'].shift(periods = 1)
    df['SMA_4_Baseline_Lag_1'] = df.groupby(level=_list[0:-1])['SMA_4_Baseline'].shift(periods = 1)
    
    return df.reset_index()


def add_last_year(df, _list):
    #list of groupby columns
    #last item in list is Calendar Week Year which is used to pull previous history (Baseline Week = Calendar Week Year) of copied dataframe
    _groupby = _list.copy()
    
    _merge_yoy = _list.copy()[0:-1]
    _merge_yoy.extend(['YOY Week'])
    
    _merge_baseline = _list.copy()[0:-1]
    _merge_baseline.extend(['Baseline Week'])
    
    df1 = df.groupby(_list, dropna = False)['LBS'].sum().reset_index()
    
    #groupby _list
    df_new = df.groupby(_list, dropna = False)['LBS'].sum().reset_index()
    
    #add week dimensions to main dataframe
    df_new = df_new.merge(TIME[['Calendar Week Year','YOY Week','Baseline Week']], how = 'left', left_on = 'Calendar Week Year', right_on = 'Calendar Week Year')
    
    df_new = df_new.merge(df1, how='left', left_on=_merge_yoy, right_on=_groupby).drop(columns={'Calendar Week Year_y'}).rename(columns={'LBS_y':'LBS_LY'})
    
    df_new = df_new.merge(df1, how='left', left_on=_merge_baseline, right_on=_groupby).drop(columns={'Calendar Week Year'}).rename(columns={
        'LBS':'LBS_Baseline','Calendar Week Year_x':'Calendar Week Year','LBS_x':'LBS'})
    
    return df_new


def add_precovid(df, _list, begin, end):
    #datefield should be last in _list
    datefield = _list[-1]
          
    #remove datefield from list
    _list = _list[0:-1]
    
    #filter data not using last and rename columns
    _df = df[(df[datefield] >= begin) & (df[datefield] <= end)].groupby(_list)['LBS'].sum() / 52
    
    return df.merge(
        _df, how = 'left', left_on = _list, right_on = _list).rename(
        columns = {'LBS_x':'LBS', 'LBS_y':'LBS_PRECOVID'}).fillna(
        value = {'LBS_PRECOVID': 0})


def add_time(df):
    df = df.merge(TIME[['Calendar Week Year','Week Starting (Sun)','Week Ending (Sat)', 'COVID Week']],
                   how = 'left', 
                   on = 'Calendar Week Year')
    
    df = df.merge(TIME[['Calendar Week Year','YOY Week','Baseline Week']], how = 'left', left_on = 'Calendar Week Year', right_on = 'Calendar Week Year')
    
    return df


def analyze_1(df, _list, begin, end):
    if 'Calendar Week Year' not in _list:
        _list.extend(['Calendar Week Year'])
    
    df = full_dataframe(df, _list)
    
    #add last year lbs
    df = add_last_year(df, _list)
    
    #add rolling calculation
    df = add_rolling(df, _list)
        
    #add preCOVID baseline
    df = add_precovid(df, _list, begin, end)
    
    df = df.round({
        'LBS' : 2,    
        'SMA_4' : 2,
        'SMA_8' : 2,
        'SMA_12' : 2,
        'LBS_LY' : 2,    
        'SMA_4_LY' : 2,
        'SMA_8_LY' : 2,
        'SMA_12_LY' : 2,
        'LBS_Baseline' : 2,    
        'SMA_4_Baseline' : 2,
        'SMA_8_Baseline' : 2,
        'SMA_12_Baseline' : 2,
        'LBS_PRECOVID' : 2,
        'LBS_Lag_1' : 2,
        'LBS_Lag_2' : 2,
        'LBS_Lag_3' : 2,
        'LBS_Lag_4' : 2,
        'LBS_Baseline_Lag_1': 2,
        'LBS_LY_Lag_1': 2,
        'SMA_4_Lag_1' : 2,
        'SMA_4_LY_Lag_1' : 2,
        'SMA_4_Baseline_Lag_1' : 2
        
    }).fillna(value = {
        'LBS' : 0,    
        'SMA_4' : 0,
        'SMA_8' : 0,
        'SMA_12' : 0,
        'LBS_LY' : 0,    
        'SMA_4_LY' : 0,
        'SMA_8_LY' : 0,
        'SMA_12_LY' : 0,
        'LBS_Baseline' : 0,    
        'SMA_4_Baseline' : 0,
        'SMA_8_Baseline' : 0,
        'SMA_12_Baseline' : 0,
        'LBS_PRECOVID' : 0,
        'LBS_Lag_1' : 0,
        'LBS_Lag_2' : 0,
        'LBS_Lag_3' : 0,
        'LBS_Lag_4' : 0,
        'LBS_Baseline_Lag_1': 2,
        'LBS_LY_Lag_1': 2,
        'SMA_4_Lag_1' : 0,
        'SMA_4_LY_Lag_1' : 0,
        'SMA_4_Baseline_Lag_1' : 0
    })
    
    return df


def save_backup(df, file_name):
    
    df.to_csv(BACKUP + file_name)
    
    return


def td_to_pandas(query, cur, title=''):
    _data = []
    _start=dt.now()
    print(dt.now().strftime('%m/%d/%Y'))
    print(f'{title} Execution started...', end='', flush=True)
    cur.execute (query)
    print(f'finished. {dt.now() - _start}', flush=True) 
    _start_fetch=dt.now()
    print(f'{title} Fetching data started...', end='', flush=True)
    for row in cur.fetchall():
        _data.append(row) 
    print(f'finished. {dt.now() - _start_fetch}', flush=True) 
    _start=dt.now()
    print(f'{title} Creating DataFrame for started...', end='', flush=True)
    _df = pd.DataFrame(_data)
    _df.columns = [x[0].replace('SAP_', '').lower() for x in cur.description]
    print(f'finished. {dt.now() - _start}', flush=True)
    return _df


def td_dataframe(select_db, query):
    with teradatasql.connect(None, 
                         host='172.29.3.43',
                         user='PNWATTERS',
                         password='teradata123') as con:
        with con.cursor() as cur:
            cur.execute (select_db)
            print('Database selected!', flush=True)            
            dim_df = td_to_pandas(query, cur, 'Query:')
            print('Dim:', dim_df.shape)
    
    return dim_df


def restaurants(df):
    #restaurants = df.loc[df['COVID Segmentation - (Restaurants)'] == 'Restaurants', :]
    
    if df.columns.isin(['COVID Segmentation - L2']).sum() > 0:
        #Rename rows
        df.loc[df['COVID Segmentation - L2'] == 'Independents (IOs) / Local Eateries / Takeaway', 'COVID Segmentation - L2'] = 'IO'
        df.loc[
            (df['COVID Segmentation - L2'] == 'All Other') | 
            (df['COVID Segmentation - L2'] == 'National Account') | 
            (df['COVID Segmentation - L2'] == 'Region Chains')| 
            (df['COVID Segmentation - L2'] == 'National Accounts'),
            'COVID Segmentation - L2'] = 'Chain'
    
    return df

def process_list(df, work_list):
    
    _process = analyze_1(df, work_list, 201910, 202009)
    
    _process = restaurants(_process)
    
    _process['Distributor'] = 'Ben E Keith'
    
    _process = add_time(_process)
    
    #for standardizing output
    work_list.extend(['Distributor','LBS','SMA_4','SMA_8','SMA_12',
                      'YOY Week','LBS_LY','SMA_4_LY','SMA_8_LY','SMA_12_LY',
                      'Baseline Week','LBS_Baseline','SMA_4_Baseline','SMA_8_Baseline','SMA_12_Baseline',
                      'LBS_Lag_1','LBS_Lag_2','LBS_Lag_3','LBS_Lag_4','LBS_Baseline_Lag_1','LBS_LY_Lag_1',
                      'SMA_4_Lag_1', 'SMA_4_LY_Lag_1', 'SMA_4_Baseline_Lag_1',
                      'LBS_PRECOVID','Week Starting (Sun)','Week Ending (Sat)','COVID Week'])
    
    if 'SKU ID' in work_list:
        #last 26 weeks in dataframe
        week_list = _process.groupby(['Calendar Week Year']).size().reset_index().drop(columns={0}).sort_values(by = 'Calendar Week Year', ascending = True).squeeze().tolist()[-26:]
        
        #filter to only the last 26 weeks
        _process = _process[_process['Calendar Week Year'].isin(week_list)]
        
    return _process[work_list]


def is_missing(df):
    #check for COVID Segmentation - L1
    missing = df[df['COVID Segmentation - L1'].isna()].groupby(['Business Unit','SIC Code','SIC Sub'], as_index = False, dropna = False)['LBS'].sum()

    if len(missing) > 0:
        print('The following segments are missing:')
        display(missing)
        missing.to_excel(DICTIONARY + 'Segments Missing Dump\\' + dt.now().strftime('%Y%m%d') + '_bek_L1_missing.xlsx', index = False)
    else:
        print(f'Nothing missing for COVID Segmentation - L1', flush = True)

    #check for product
    missing = df[df['Consolidated Category'].isna()].groupby(['Group','Family','SKU ID','Prod Nbr','Product',
                                                                        'Product Ext.Description','Pack / Size'], as_index = False, dropna = False)['LBS'].sum()

    if len(missing) > 0:
        print('The following products are missing:')
        display(missing)
        missing.to_excel(DICTIONARY + 'Segments Missing Dump\\' + dt.now().strftime('%Y%m%d') + r'_bek_missing.xlsx', index = False)
    else:
        print(f'Nothing missing for Product', flush = True)
        
def full_dataframe(df, _list):
    weeks = df.groupby(['Calendar Week Year']).size().reset_index().drop(columns={0})
    segments = df.groupby(_list[0:-1]).size().reset_index().drop(columns={0})
    
    _df = segments.assign(key=1).merge(weeks.assign(key=1), how='outer', on='key').drop(columns = {'key'}) 
    
    return _df.merge(df, how = 'left', on = _list) 

def clean_city(df):
    df['City'] = df['City'].str.strip()
    df['City'] = df['City'].str.upper()
    df['City'].fillna('NA', inplace = True)
    
    #cities = 'TORONTO|MONTREAL|OTTAWA|CALGARY|VANCOUVER|WINNIPEG|MONTREAL|HAMILTON|HALIFAX'
    cities = 'NOT USED CURRENTLY'
    
    #change each city name to the name of the city that matches, cleans up the city names
    for c in cities.split('|'):
        df.loc[df['City'].str.match(c), 'City'] = c
    
    #change all other cities to NA
    df.loc[~df['City'].str.match(cities), 'City'] = 'NA'
    
    return df

### 5. Sell-in vs. Sell-out
Run cell below

In [5]:
def teradata_sales(sellout):
    #SET QUERY_BAND = 'ApplicationName=MicroStrategy;Version=9.0;ClientUser=NEWATTER;Source=Vantage; Action=BEK Performance;StartTime=20200901T101924;JobID=55096;Importance=666;'  FOR SESSION;
    
    #the current week is pulled from the time dictionary table
    to_week = int(TIME[(TIME['Week Starting (Mon)'] <= dt.now()) & (TIME['Week Ending (Sun)'] >= dt.now())]['Calendar Week Year'].values)
    
    print(f'Starting Teradata connect...', flush = True)
    
    select_db = "DATABASE DL_GBL_TAS_BI"

    query = '''
    select a14.FISCAL_WEEK_NUMBER as FISCAL_WEEK_NUMBER,
        (a14.FISCAL_WEEK_NUMBER_DESCR || ' ' || a14.START_DATE_OF_SAPYW) as FISCAL_WEEK,
        a14.CALENDAR_WEEK_NAME as CALENDAR_WEEK_NUMBER,
        (a14.CALENDAR_WEEK_LONG_DESCRIPTION || ' ' || a14.START_DATE_OF_SAPYW) as CALENDAR_WEEK,
        RIGHT(a16.CUSTOMER_HIER_LVL_1,CAST(10 AS INTEGER)) as CUSTOMER_HIER_LVL_1,
        a16.CUSTOMER_HIER_LVL_1_NAME as CUSTOMER_HIER_LVL_1_NAME,
        a17.DIVISION_NAME as DIVISION_NAME,
        a12.CATEGORY_DESC as CATEGORY_DESC,
        a12.SUB_CATEGORY_DESC as SUB_CATEGORY_DESC,
        a13.PRODUCT_GROUP_FORMAT_DESC as L1_PRODUCT_HIERARCHY,
        a13.PRODUCT_GROUP_SUB_FORMAT_DESC as L2_PRODUCT_HIERARCHY,
        a15.MATERIAL_PRICING_GROUP_ID as MATERIAL_PRICING_GROUP_ID,
        a18.MATERIAL_PRICING_GROUP_DESCRIPTION as MATERIAL_PRICING_GROUP_DESCRIPTION,
        TRIM (LEADING '0' FROM a13.MATERIAL_ID) as MATERIAL_ID,
        a13.MATERIAL_DESCRIPTION as MATERIAL_NAME,
        sum(a11.SALES_VOLUME_WEIGHT_LBS) as ACTUAL_VOLUME_LBS
    from DL_GBL_TAS_BI.FACT_SALES_ACTUAL as a11
    join DL_GBL_TAS_BI.VW_H_PRODUCT_ALL_SALES as a12
        on (a11.MATERIAL_ID = a12.MATERIAL_ID)
    join DL_GBL_TAS_BI.D_MATERIAL_DN_ALL as a13
        on (a11.MATERIAL_ID = a13.MATERIAL_ID)
    join DL_GBL_TAS_BI.D_TIME_FY_V6 as a14
        on (a11.ACCOUNTING_PERIOD_DATE = a14.DAY_CALENDAR_DATE)
    join DL_GBL_TAS_BI.D_MATERIAL_SALES_DATA as a15
        on (a11.DISTRIBUTION_CHANNEL_ID = a15.DISTRIBUTION_CHANNEL_ID and 
        a11.MATERIAL_ID = a15.MATERIAL_ID and 
        a11.SALES_ORGANISATION_ID = a15.SALES_ORGANISATION_ID)
    join DL_GBL_TAS_BI.VW_H_CUSTOMER_ALL_DIVISION00 as a16
        on (a11.CUSTOMER_ID = a16.CUSTOMER and 
        a11.DISTRIBUTION_CHANNEL_ID = a16.DISTRIBUTION_CHANNEL and 
        a11.SALES_ORGANISATION_ID = a16.SALES_ORGANISATION)
    join DL_GBL_TAS_BI.D_DIVISION as a17
        on (a13.DIVISION_ID = a17.DIVISION_ID)
    join DL_GBL_TAS_BI.D_MATERIAL_PRICING_GROUP as a18
        on (a15.MATERIAL_PRICING_GROUP_ID = a18.MATERIAL_PRICING_GROUP_ID)
    where (a14.FISCAL_YEAR_CODE in ('FY2019', 'FY2020', 'FY2021','FY2022')
        and a11.SALES_ORGANISATION_ID in ('US01')
        and a11.DISTRIBUTION_CHANNEL_ID in ('10')
        and RIGHT(a16.CUSTOMER_HIER_LVL_1,CAST(10 AS INTEGER)) in ('6500002764'))
        and a14.CALENDAR_WEEK_NAME < ''' + str(to_week) + ''' 
    group by a14.FISCAL_WEEK_NUMBER,
        (a14.FISCAL_WEEK_NUMBER_DESCR || ' ' || a14.START_DATE_OF_SAPYW),
        a14.CALENDAR_WEEK_NAME,
        (a14.CALENDAR_WEEK_LONG_DESCRIPTION || ' ' || a14.START_DATE_OF_SAPYW),
        RIGHT(a16.CUSTOMER_HIER_LVL_1,CAST(10 AS INTEGER)),
        a16.CUSTOMER_HIER_LVL_1_NAME,
        a17.DIVISION_NAME,
        a12.CATEGORY_DESC,
        a12.SUB_CATEGORY_DESC,
        a13.PRODUCT_GROUP_FORMAT_DESC,
        a13.PRODUCT_GROUP_SUB_FORMAT_DESC,
        a15.MATERIAL_PRICING_GROUP_ID,
        a18.MATERIAL_PRICING_GROUP_DESCRIPTION,
        TRIM (LEADING '0' FROM a13.MATERIAL_ID),
        a13.MATERIAL_DESCRIPTION
    ;'''

    #create dataframe using both functions td_to_pandas and td_dataframe
    df = td_dataframe(select_db, query)
    
    return teradata_transform(df, sellout)


def teradata_transform(sellin, sellout):
    #consolidates teradata sales with sellout data
    
    #convert from object datatype to float (exports as a number instead of string)
    sellin['actual_volume_lbs'] = sellin['actual_volume_lbs'].astype('float64')
    
    #rename columns for consistancy
    sellin = sellin.rename(columns = {'actual_volume_lbs':'LBS', 
                                      'calendar_week_number':'Calendar Week Year',
                                      'l1_product_hierarchy':'L1 Product Hierarchy',
                                      'l2_product_hierarchy':'L2 Product Hierarchy'})
    
    #transform calendar week year from teradata
    sellin['Calendar Week Year'] = pd.to_numeric(sellin['Calendar Week Year'], errors = 'coerce')

    #transform category so its consolidated
    sellin['Consolidated Category'] = sellin['category_desc']
    sellin.loc[sellin['Consolidated Category'] == 'Sweet Potato' , 'Consolidated Category'] = 'Potato'
    sellin.loc[sellin['Consolidated Category'] != 'Potato' , 'Consolidated Category'] = 'Prepared Foods'
    
    #analyze sellin data
    #sellin = analyze_1(sellin, ['Consolidated Category', 'L1 Product Hierarchy','L2 Product Hierarchy'], 201910, 202009)
    #analyze sellin data
    sellin = analyze_1(sellin, ['Consolidated Category'], 201910, 202009)
    
    '''
    #   Column                 Non-Null Count  Dtype  
    ---  ------                 --------------  -----  
     0   Consolidated Category  296 non-null    object 
     1   Calendar Week Year     296 non-null    int64  
     2   MCCAIN LBS             296 non-null    float64
     3   MCCAIN SMA_4           296 non-null    float64
     4   MCCAIN SMA_8           296 non-null    float64
     5   MCCAIN SMA_12          296 non-null    float64
     6   LBS_Lag_1              296 non-null    float64
     7   LBS_Lag_2              296 non-null    float64
     8   LBS_Lag_3              296 non-null    float64
     9   LBS_Lag_4              296 non-null    float64
     10  YOY Week               242 non-null    float64
     11  Baseline Week          242 non-null    float64
     12  LBS_LY                 296 non-null    float64
     13  SMA_4_LY               296 non-null    float64
     14  SMA_8_LY               296 non-null    float64
     15  SMA_12_LY              296 non-null    float64
     16  LBS_Baseline           296 non-null    float64
     17  SMA_4_Baseline         296 non-null    float64
     18  SMA_8_Baseline         296 non-null    float64
     19  SMA_12_Baseline        296 non-null    float64
     20  MCCAIN PRECOVID        296 non-null    float64
    '''
    
    #rename columns accordingly
    sellin = sellin.rename(columns = {'LBS':'MCCAIN LBS',
                                      'SMA_4':'MCCAIN SMA_4',
                                      'SMA_8':'MCCAIN SMA_8',
                                      'SMA_12':'MCCAIN SMA_12',
                                      'LBS_PRECOVID':'MCCAIN PRECOVID',
                                      'LBS_Lag_1':'MCCAIN Lag_1',
                                      'LBS_Lag_2':'MCCAIN Lag_2',
                                      'LBS_Lag_3':'MCCAIN Lag_3',
                                      'LBS_Lag_4':'MCCAIN Lag_4',
                                      'LBS_Baseline' : 'MCCAIN LBS_Baseline',
                                      'SMA_4_Baseline' : 'MCCAIN SMA_4_Baseline',
                                      'SMA_8_Baseline' : 'MCCAIN SMA_8_Baseline',
                                      'SMA_12_Baseline' : 'MCCAIN SMA_12_Baseline',
                                      'SMA_4_Lag_1':'MCCAIN SMA_4_Lag_1',
                                      'SMA_4_Baseline_Lag_1' : 'MCCAIN SMA_4_Baseline_Lag_1',
                                      'LBS_Baseline_Lag_1': 'MCCAIN LBS_Baseline_Lag_1'
                                     })
    
    '''
    #   Column                 Non-Null Count  Dtype  
    ---  ------                 --------------  -----  
     0   Consolidated Category  242 non-null    object 
     1   Calendar Week Year     242 non-null    int64  
     2   LBS                    242 non-null    float64
     3   SMA_4                  242 non-null    float64
     4   SMA_8                  242 non-null    float64
     5   SMA_12                 242 non-null    float64
     6   LBS_Lag_1              242 non-null    float64
     7   LBS_Lag_2              242 non-null    float64
     8   LBS_Lag_3              242 non-null    float64
     9   LBS_Lag_4              242 non-null    float64
     10  YOY Week               242 non-null    int64  
     11  Baseline Week          242 non-null    int64  
     12  LBS_LY                 242 non-null    float64
     13  SMA_4_LY               242 non-null    float64
     14  SMA_8_LY               242 non-null    float64
     15  SMA_12_LY              242 non-null    float64
     16  LBS_Baseline           242 non-null    float64
     17  SMA_4_Baseline         242 non-null    float64
     18  SMA_8_Baseline         242 non-null    float64
     19  SMA_12_Baseline        242 non-null    float64
     20  LBS_PRECOVID           242 non-null    float64
    '''
    
    #analyze sellout data
    df = analyze_1(sellout, ['Consolidated Category'], 201910, 202009)
    
    
    df = df.merge(sellin[['Calendar Week Year','Consolidated Category','MCCAIN LBS','MCCAIN SMA_4','MCCAIN SMA_8','MCCAIN SMA_12','MCCAIN PRECOVID',
                          'MCCAIN LBS_Baseline','MCCAIN SMA_4_Baseline','MCCAIN SMA_8_Baseline','MCCAIN SMA_12_Baseline',
                          'MCCAIN Lag_1', 'MCCAIN Lag_2', 'MCCAIN Lag_3', 'MCCAIN Lag_4','MCCAIN LBS_Baseline_Lag_1',
                          'MCCAIN SMA_4_Lag_1', 'MCCAIN SMA_4_Baseline_Lag_1']], how = 'left', 
                  left_on = ['Calendar Week Year','Consolidated Category'], right_on = ['Calendar Week Year','Consolidated Category'])
    
    df = df.fillna({'MCCAIN LBS': 0,
                    'MCCAIN SMA_4': 0,
                    'MCCAIN SMA_8': 0,
                    'MCCAIN SMA_12': 0,
                    'MCCAIN PRECOVID': 0,
                    'MCCAIN Lag_1': 0,
                    'MCCAIN Lag_2': 0,
                    'MCCAIN Lag_3': 0,
                    'MCCAIN Lag_4': 0,
                    'MCCAIN LBS_Baseline': 0,
                    'MCCAIN SMA_4_Baseline': 0,
                    'MCCAIN SMA_8_Baseline': 0,
                    'MCCAIN SMA_12_Baseline': 0,
                    'MCCAIN LBS_Baseline_Lag_1':0,
                    'MCCAIN SMA_4_Lag_1' : 0,
                    'MCCAIN SMA_4_Baseline_Lag_1' : 0
                   })
    
    df['Distributor'] = 'Ben E Keith'

    df = add_time(df)
    
    df = df[['Consolidated Category','Distributor','Calendar Week Year',
             'LBS','SMA_4','SMA_8','SMA_12','LBS_PRECOVID',
             'LBS_Baseline','SMA_4_Baseline','SMA_8_Baseline','SMA_12_Baseline',
             'LBS_Lag_1', 'LBS_Lag_2', 'LBS_Lag_3', 'LBS_Lag_4', 'LBS_Baseline_Lag_1', 'SMA_4_Lag_1', 'SMA_4_Baseline_Lag_1',
             'MCCAIN LBS','MCCAIN SMA_4','MCCAIN SMA_8','MCCAIN SMA_12','MCCAIN PRECOVID',
             'MCCAIN LBS_Baseline','MCCAIN SMA_4_Baseline','MCCAIN SMA_8_Baseline','MCCAIN SMA_12_Baseline',
             'MCCAIN Lag_1', 'MCCAIN Lag_2', 'MCCAIN Lag_3', 'MCCAIN Lag_4','MCCAIN LBS_Baseline_Lag_1','MCCAIN SMA_4_Lag_1','MCCAIN SMA_4_Baseline_Lag_1',
             'Week Starting (Sun)','Week Ending (Sat)','COVID Week']]
    
    return df

### 6. Import Raw Data
Run cell below

In [6]:
'''
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177665 entries, 0 to 177664
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Branch                   177665 non-null  object 
 1   Business Unit            177664 non-null  object 
 2   SIC Code                 177664 non-null  object 
 3   SIC Sub                  177664 non-null  object 
 4   Customer Nbr.            177664 non-null  float64
 5   Customer Name            177664 non-null  object 
 6   Customer Address1        177664 non-null  object 
 7   Customer Address2        8978 non-null    object 
 8   Customer City            177661 non-null  object 
 9   Customer State           177664 non-null  object 
 10  Customer Zip             177664 non-null  float64
 11  Family                   177664 non-null  object 
 12  Brand                    177664 non-null  object 
 13  Group                    177664 non-null  object 
 14  Manufacture Prod.Nbr.    177664 non-null  object 
 15  Prod Nbr                 177664 non-null  float64
 16  Product                  177664 non-null  object 
 17  Pack / Size              177664 non-null  object 
 18  Product Ext.Description  154392 non-null  object 
 19  Week of                  177664 non-null  object 
 20  Month                    177664 non-null  object 
 21  LBS                      177665 non-null  object 
dtypes: float64(3), object(19)
memory usage: 29.8+ MB
'''

#new file name in historical file directory
_new = import_file(PATH + 'BEK Update.xls')

print(f'Imported {_new.shape[0]} records', flush = True)

Sheet1


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_24516\4131771016.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(add)


Sheet2


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_24516\4131771016.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(add)


Sheet3
Imported 177038 records


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_24516\4131771016.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(add)


### 7. Apply Dictionary to Raw Data / Check For Missing Segmentation
Run cell below

In [7]:
#add dictionary to new data
_new_df = apply_dictionary(_new, 'BEK - US.xlsx')

#look for missing segmentation
is_missing(_new_df)

Shape before adding dictionary: (177038, 22)
Shape after adding segmentation: (177038, 28)
Shape after adding product segmentation: (177038, 31)
Shape before adding time: (177038, 32)
Shape after adding time: (177038, 33)
Shape after adding dictionary: (177034, 34)
Nothing missing for COVID Segmentation - L1
Nothing missing for Product


### 8. Combine New and Base Datasets
Run cell below

In [8]:
_import_df = pd.read_csv(BACKUP + 'BEK.csv', low_memory = False, thousands = ',', decimal = '.', dtype = {
            'Calendar Week Year':np.int64,
            'LBS':np.float64})

print(f'Imported shape...{_import_df.shape}', flush = True)

#create list to exclude from base data
#exclude_list = _new_df.groupby('Calendar Week Year').size().reset_index().drop(columns={0}).squeeze().to_list()

#create unique list values
exclude_list = list(dict.fromkeys(_new_df['Calendar Week Year'].values.squeeze().tolist()))

#turn list to string
include = str(exclude_list)[1:-1]

#import all records from base data minus the new data
_base = _import_df[~_import_df['Calendar Week Year'].isin(exclude_list)][['City','State','State Name','COVID Segmentation - L1','COVID Segmentation - L2',
                 'COVID Segmentation - (Restaurants)','COVID Segmentation - (Restaurants: Sub-Segment)','Restaurant Service Type',
                 'Consolidated Category','SKU ID','Cuisine Type','L1 Product Hierarchy','L2 Product Hierarchy','Calendar Week Year','LBS']].append(
        _new_df[['City','State','State Name','COVID Segmentation - L1','COVID Segmentation - L2',
                 'COVID Segmentation - (Restaurants)','COVID Segmentation - (Restaurants: Sub-Segment)','Restaurant Service Type',
                 'Consolidated Category','SKU ID','Cuisine Type','L1 Product Hierarchy','L2 Product Hierarchy','Calendar Week Year','LBS']])


#clean up city column
_base = clean_city(_base)

#create a copy but just for restaurants#
#_restaurants = restaurants(_base)

print(f'Final shape...{_base.shape}', flush = True)
#print(f'Restaurants shape...{_restaurants.shape}', flush = True)


Imported shape...(2113771, 16)


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_24516\943055581.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _base = _import_df[~_import_df['Calendar Week Year'].isin(exclude_list)][['City','State','State Name','COVID Segmentation - L1','COVID Segmentation - L2',


Final shape...(2126107, 15)


### 9. Execute Analysis
Run cell below

In [9]:

_list = []

#Output 1: COVID L1 - List 0
_list.append(['City','State Name','COVID Segmentation - L1','COVID Segmentation - L2','Restaurant Service Type','Consolidated Category'])

#Output 2: COVID L1 - List 1
_list.append(['State Name','COVID Segmentation - L1','COVID Segmentation - L2','Restaurant Service Type','SKU ID','Consolidated Category','L1 Product Hierarchy','L2 Product Hierarchy'])

#Create dataframes
print(f'Processing Segments', flush = True)
output1 = process_list(_base, _list[0])

print(f'Processing Sell in vs Sell out', flush = True)
output2 = teradata_sales(_base)

#print(f'Processing SKU', flush = True)
#output3 = process_list(_base, _list[1])

#Output 2: COVID L1 - List 1
#_list.append(['COVID Segmentation - L1','COVID Segmentation - L2','Restaurant Service Type','Consolidated Category','L1 Product Hierarchy','L2 Product Hierarchy'])
#output4 = process_list(_base, _list[2])

print('All done')



Processing Segments
Processing Sell in vs Sell out
Starting Teradata connect...
Database selected!
07/07/2022
Query: Execution started...finished. 0:01:16.042287
Query: Fetching data started...finished. 0:00:11.625336
Query: Creating DataFrame for started...finished. 0:00:00.055248
Dim: (34180, 16)
All done


### 10. Upload Analysis to Teradata
Run cell below

In [10]:
def td_upload(select_db, df, table_name):
    with teradatasql.connect(None, 
                         host='172.29.3.43',
                         user='PNWATTERS',
                         password='teradata123') as con:
        with con.cursor() as cur:
            cur.execute (select_db)
            d = dt.now().strftime('%m/%d/%Y %H:%M:%S %p')
            print(f'Database selected! {d}', flush=True)            

            delete_from_td(df, table_name, cur)
            insert_into_td(df, table_name, cur)

def delete_from_td(df, table_name, cur):
    distributor = df.groupby('Distributor').size().reset_index().drop(columns=0).to_numpy()[0][0]
    
    print(f'Deleting records for: {distributor} in table: {table_name}', flush = True)          
    
    query = '''
    DELETE FROM ''' + table_name  + ''' 
    WHERE "Distributor" = ''' + "'" + distributor + "'" + ''' AND "Calendar Week Year" IN (''' + include + ")"
    
    #query = '''
    #DELETE FROM ''' + table_name  + ''' 
    #WHERE "Distributor" = ''' + "'" + distributor + "'"
    
    cur.execute (query)
    
def insert_into_td(df, table_name, cur):
    insert_list = df.values.tolist()
    
    #creates ?, ?,.... string used in query for teradata fastload
    insert_columns = ('?, ' * len(df.columns)).rstrip(', ')
    
    print(f'Inserting records into {table_name}', flush = True)
    
    query = "INSERT INTO " + table_name  + " (" + insert_columns + ")"
    #query = "{fn teradata_try_fastload}INSERT INTO " + table_name  + " (" + insert_columns + ")"
    
    cur.execute (query, insert_list)
    
    print(f'Inserted {df.shape[0]} records', flush = True)
    

select_db = 'DATABASE DL_NA_PROTOTYPING'

#only send new data
td_upload(select_db, output1[output1['Calendar Week Year'].isin(exclude_list)], 'SELLOUT_REGION')
td_upload(select_db, output2[output2['Calendar Week Year'].isin(exclude_list)], 'SELLOUT_AND_SELLIN')

#only send all data
#td_upload(select_db, output1, 'SELLOUT_REGION')
#td_upload(select_db, output2, 'SELLOUT_AND_SELLIN')

#td_upload(select_db, output3[output3['Calendar Week Year'].isin(exclude_list)].astype({'SKU ID':'str'}), 'SELLOUT_REGION_SKU')

Database selected! 07/07/2022 15:34:28 PM
Deleting records for: Ben E Keith in table: SELLOUT_REGION
Inserting records into SELLOUT_REGION
Inserted 8470 records
Database selected! 07/07/2022 15:34:38 PM
Deleting records for: Ben E Keith in table: SELLOUT_AND_SELLIN
Inserting records into SELLOUT_AND_SELLIN
Inserted 28 records


### Save Backup When Ready

In [11]:
save_backup(_base, 'BEK.csv')